In [ ]:
from music21 import *
import numpy as np
import glob
import math
import os
from cntk import Trainer, Axis
from cntk.learner import adam_sgd, momentum_sgd, momentum_as_time_constant_schedule, learning_rate_schedule, UnitType
from cntk.ops import input_variable, cross_entropy_with_softmax, classification_error, binary_cross_entropy, times_transpose, log, squared_error
from cntk.persist import load_model, save_model
from cntk.blocks import LSTM, Stabilizer
from cntk.layers import Recurrence, Dense, Dropout, BatchNormalization
from cntk.utils import get_train_eval_criterion, get_train_loss
from cntk.device import set_default_device, gpu

path = "midiFiles"

In [2]:
lowerBound = 24 # = [python] index 0 ([R] 1. possision)
upperBound = 102 # = first index out of bound => len =78
nNotes = upperBound-lowerBound
notesDim = nNotes*2
hidden_dim = nNotes
num_layers = 2


batch_len = 16*8 # length of each sequence

def loadPieces(midiFiles):
    pieces = {}
    totalLength= len(midiFiles)
    fnameNumber=0
    for fname in midiFiles:
        fnameNumber+=1
        if fname[-4:] not in ('.mid','.MID'):
            continue
        isNotBackSlash=1
        bIndex=0
        while(isNotBackSlash):
            bIndex-=1
            if fname[bIndex]=='\\':
                bIndex+=1
                isNotBackSlash=False
        name = fname[bIndex:-4]
        print("Loading",name,"...",fnameNumber,'/',totalLength)
        outMatrix = midi2MusicMinMatrix(fname)
        if len(outMatrix) < batch_len:
            print("     Loaded", name,'failed')
            continue
        else:
            print("     Loaded", name,'done')
        pieces[name] = outMatrix
    return(pieces)



def midi2MusicMinMatrix(parseLink):
    midiFile = converter.parse(parseLink)
    #if (midiFile.flat.getTimeSignatures().timeSignature.numerator not in (2,4)) or (midiFile.flat.getTimeSignatures().timeSignature.denominator not in (2,4)):
    #	print('     Track: is not 4/4 or similar')
    #	return(0)	
    if (midiFile.flat.getTimeSignatures().timeSignature.numerator not in (2,4)) or (midiFile.flat.getTimeSignatures().timeSignature.denominator not in (2,4)):
        print('     Track: is not 4/4')
        printing = '     Track is '+str(midiFile.flat.getTimeSignatures().timeSignature.numerator)+' / ' + str(midiFile.flat.getTimeSignatures().timeSignature.denominator)
        print(printing)
        return(0)	
    midiPartLen=len(midiFile)
    maxTime = midiFile.highestTime
    outMatrix = [[[0,0] for tang in range(upperBound-lowerBound)]+[[lenTimes16/4]] for lenTimes16 in range(math.ceil(maxTime*4+2))]
    # outMatrix = [[[0,0] for tang in range(upperBound-lowerBound)]+[lenTimes16/4] for lenTimes16 in range(math.ceil(maxTime*4+2))]
    for t in range(midiPartLen):
        try:
            instrument=midiFile[t].getInstrument(returnDefault=False).instrumentName
        except: 
            instrument='Piano'
        #if not instrument:
        #	instrument='Piano'
        #if instrument!='Piano':
        #	print('          Part',t,'/',midiPartLen-1,' dropped, instrument was not piano')
        #	continue			
        notes=midiFile[t].flat.notes
        for i in range(0,len(notes)):
            if not notes[i].isChord:
                if (notes[i].pitch.midi<lowerBound) or (notes[i].pitch.midi>=upperBound):
                    continue
                timeIndex=round(notes[i].offset*4)+1
                notePitch=notes[i].pitch.midi-lowerBound
                outMatrix[timeIndex][notePitch]=[1,1]
                for holdLen in range(1,round(notes[i].duration.quarterLength*4)):
                    outMatrix[timeIndex+holdLen][notePitch][1]=1
            else:
                timeIndex=round(notes[i].offset*4)+1
                duration=round(notes[i].duration.quarterLength*4)+1
                for j in range(len(notes[i])):
                    if (notes[i][j].pitch.midi<lowerBound) or (notes[i][j].pitch.midi>=upperBound):
                        continue
                    #timeIndex=round(notes[i][j].offset*4)+1
                    notePitch=notes[i][j].pitch.midi-lowerBound
                    outMatrix[timeIndex][notePitch]=[1,1]
                    for holdLen in range(1,duration):
                        outMatrix[timeIndex+holdLen][notePitch][1]=1	
    return(outMatrix)

def MusicMinMatrix2midi(MusicMinMatrixPart,S,bpm=100):
    #S = stream.Score()
    nTime=len(MusicMinMatrixPart)
    S.insert(0,tempo.MetronomeMark(number=bpm))
    p1 = stream.Part()
    p1.id = 'part1'
    nCons=0
    thisIndex=0
    mLen = len(MusicMinMatrixPart[0])
    for j in range(mLen-1):
        for i in range(nTime):
            if (nCons!=0) and (MusicMinMatrixPart[i][j] in ([0,0],[1,1])):
                n.duration.quarterLength = 0.25*nCons
                p1.insert(MusicMinMatrixPart[thisIndex][mLen-1][0],n)
                nCons=0
            if MusicMinMatrixPart[i][j]==[1,1]:
                n = note.Note()
                n.midi = lowerBound+j
                thisIndex = i
                nCons+=1
            elif (MusicMinMatrixPart[i][j]==[0,1]) and (nCons!=0):
                nCons+=1
        if (nCons != 0):
            n.duration.quarterLength = 0.25*nCons
            p1.insert(MusicMinMatrixPart[thisIndex][mLen-1][0],n)
            nCons=0
    S.insert(0,p1)
    
    
def Piece2Data(Pieces):
    tempData = sum(list(Pieces.values()),[])
    data=[sum(timePart,[]) for timePart in tempData]
    return(np.array(data,dtype=np.float32))
    

def get_data(p, minibatch_size, data):
    x = data[p:p+minibatch_size,0:156]
    y = data[p+1:p+minibatch_size+1,0:156]
    #xsum=x.sum(axis=1, keepdims=True)
    
    X = x/np.maximum(x.sum(axis=1, keepdims=True),1)
    Y = y/np.maximum(y.sum(axis=1, keepdims=True),1)
    return([X],[Y])

def data2MusicMatric(data):
    timeIndex=0
    MusicMatrix=[0]*len(data)
    MusicMatrixRow=[0]*(upperBound-lowerBound)
    for i in range(len(data)):
        for j in range(upperBound-lowerBound):
            MusicMatrixRow[j] = [int(data[i,j*2]),int(data[i,j*2+1])]
        MusicMatrix[i] = MusicMatrixRow+[[timeIndex]]
        timeIndex+=0.25
    return(MusicMatrix)



def Output2data(p,threshold=1.11):
    p=p[0][0]
    return(list((p>p.mean()+threshold)*1))

# prime_data=data[5:20,:]
def sample(z,prime_data=np.array([[0]*((upperBound-lowerBound)*2)],dtype=np.float32),length=300,threshold=1.11):
    output=np.zeros((length, 156), dtype=np.float32)
    if len(prime_data[0])!=156:
        prime_data[:,0:156]
    randomNumb=np.random.choice(range(78))
    if len(prime_data)==1: 
        prime_data[0,randomNumb*2] = 1
        prime_data[0,randomNumb*2+1] = 1
    for i in range(len(prime_data)):
        x=prime_data[i]
        inputen=[np.array([x/np.maximum(x.sum(),1)],dtype=np.float32)]
        if i==0:
            arguments=(inputen,[True])
        else:
            arguments=(inputen,[False])
        p = z.eval(arguments)
        
    x=prime_data[len(prime_data)-1]
    
    for i in range(length):
        inputen=[np.array([x/np.maximum(x.sum(),1)],dtype=np.float32)]
        arguments=(inputen,[False])
        p = z.eval(arguments)
        output[i,:]=Output2data(p,threshold=threshold)
        x=output[i,:]
    return(output)#return([list(testout[i]) for i in range(300)])

In [3]:
midiFiles = glob.glob(path + '/*.mid')
Pieces = loadPieces(midiFiles)
data = Piece2Data(Pieces)

Loading jingbellPiano ... 1 / 1
     Loaded jingbellPiano done


In [4]:
def train_mm(data):

    # create the stabilizer function from blocks
    stabilize = Stabilizer()
    data_size=len(data)

    # load the data and vocab
    # data, char_to_ix, ix_to_char, data_size, vocab_dim = load_data_and_vocab(training_file)

    # Source and target inputs to the model
    batch_axis = Axis.default_batch_axis()
    input_seq_axis = Axis('inputAxis')

    input_dynamic_axes = [batch_axis, input_seq_axis]
    raw_input = input_variable(shape=(notesDim), dynamic_axes=input_dynamic_axes)
    raw_labels = input_variable(shape=(notesDim), dynamic_axes=input_dynamic_axes)

    input_sequence = raw_input
    label_sequence = raw_labels

    # LSTM
    encoder_output = stabilize(input_sequence)
    for i in range(0, num_layers):
        encoder_output = Recurrence(LSTM(hidden_dim, enable_self_stabilization=True)) (encoder_output.output)
        #encoder_output = BatchNormalization() (encoder_output.output)
        encoder_output = Dropout(0.3) (encoder_output.output)

    # get output of the LSTM
    states = encoder_output.output

    # dense layer    
    z = Dense(notesDim) (states)

    # ce = binary_cross_entropy(z, label_sequence)
    ce = cross_entropy_with_softmax(z, label_sequence)
    # ce = times_transpose(label_sequence,log(z))
    # ce = squared_error(z,label_sequence)
    # errs = squared_error(z,label_sequence)
    errs = classification_error(z, label_sequence)

    # Instantiate the trainer object to drive the model training
    lr_per_sample = learning_rate_schedule(0.001, UnitType.sample)
    momentum_time_constant = momentum_as_time_constant_schedule(1100)
    clipping_threshold_per_sample = 5.0
    gradient_clipping_with_truncation = True
    learner = adam_sgd(z.parameters, lr_per_sample, momentum_time_constant, 
                           gradient_clipping_threshold_per_sample=clipping_threshold_per_sample,
                           gradient_clipping_with_truncation=gradient_clipping_with_truncation)
    trainer = Trainer(z, ce, errs, learner)
    
    minibatch_size = 10 
    training_progress_output_freq = 200
    sample_freq = 20
    epochs = 50
    minibatches_per_epoch = int((data_size / minibatch_size))
    minibatches = epochs * minibatches_per_epoch
    
    e = 0
    p = 0
    for i in range(0, minibatches):

        if p + minibatch_size+1 >= data_size:
            p = 0
            e += 1
            model_filename = "models/deepjingling-composer_epoch%d.dnn" % e
            save_model(z, model_filename)
            print("Saved model to '%s'" % model_filename)

        # get the data            
        features, labels = get_data(p, minibatch_size, data)

        # Specify the mapping of input variables in the model to actual minibatch data to be trained with
        # If it's the start of the data, we specify that we are looking at a new sequence (True)
        mask = [False] 
        if p == 0:
            mask = [True]
        arguments = ({raw_input : features, raw_labels : labels}, mask)
        trainer.train_minibatch(arguments)

        if i % training_progress_output_freq == 0:
            print("Minibatch: {}, Train Loss: {}, Train Evaluation Criterion: {}".format(i,
                      get_train_loss(trainer), get_train_eval_criterion(trainer)))
            print("Epoch %d, %f %% done" % (e, ((float(i) / float(minibatches_per_epoch)) - e) * 100.0))
        
        #if i % sample_freq == 0:
        #    print('Sample:')
        #    sample(z, vocab_dim)

        p += minibatch_size
    return(z)

In [5]:
z=train_mm(data)

Minibatch: 0, Train Loss: 5.049293899536133, Train Evaluation Criterion: 1.0
Epoch 0, 0.000000 % done
Saved model to 'models/deepjingling-composer_epoch1.dnn'
Minibatch: 200, Train Loss: 4.526654815673828, Train Evaluation Criterion: 0.7625
Epoch 1, 96.078431 % done
Saved model to 'models/deepjingling-composer_epoch2.dnn'
Saved model to 'models/deepjingling-composer_epoch3.dnn'
Minibatch: 400, Train Loss: 3.151409721374512, Train Evaluation Criterion: 0.9375
Epoch 3, 92.156863 % done
Saved model to 'models/deepjingling-composer_epoch4.dnn'
Saved model to 'models/deepjingling-composer_epoch5.dnn'
Minibatch: 600, Train Loss: 2.5927192687988283, Train Evaluation Criterion: 0.5625
Epoch 5, 88.235294 % done
Saved model to 'models/deepjingling-composer_epoch6.dnn'
Saved model to 'models/deepjingling-composer_epoch7.dnn'
Minibatch: 800, Train Loss: 3.2467010498046873, Train Evaluation Criterion: 0.6125
Epoch 7, 84.313725 % done
Saved model to 'models/deepjingling-composer_epoch8.dnn'
Saved mo

In [6]:
#prime_data=np.array([[0]*((upperBound-lowerBound)*2)],dtype=np.float32)
#print(prime_data)
#len(prime_data[0])
#print(len(prime_data))
#data[0]
#data[1:3]
testout=sample(z,prime_data=np.array([[0]*((upperBound-lowerBound)*2)],dtype=np.float32),length=300,threshold=5)
musicM=data2MusicMatric(testout)
S = stream.Score()
MusicMinMatrix2midi(musicM,S,100)
sp=midi.realtime.StreamPlayer(S)
sp.play()

In [ ]:
S.write('midi', 'notFirstMusic') 